This the notebook where a whole sample is loaded and depending on the information of extracted roi visualize the roi squares on top of the sample slices 

In [1]:
import os 
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import ipywidgets as widgets

import tifffile

from poreUtils import *

In [2]:
# Path to the slices 
# data_dir = 'E:\\Data\\sam_data\\new\\MD_1264_B10_Z0.0mm\\slices'
data_dir = 'F:\\MD_1264_A16_Z3.3mm_corr_phrt\\slices'
tiffs = os.listdir(data_dir)

In [3]:
# Reading all the slices in a Volume 
vol = np.empty(shape=(1700, 3686, 3686), dtype=np.uint8)
for i, fname in enumerate(tqdm(tiffs)):
    im = Image.open(os.path.join(data_dir, fname))
    imarray = np.array(im)
    imarray = np.clip(imarray, 0.0005, 0.003)
    imarray = norm8bit(imarray)
    vol[i, :, :] = imarray

  0%|          | 0/1700 [00:00<?, ?it/s]

e:\projects\poreAnalysis\poreUtils.py:127: RuntimeWarning: invalid value encountered in subtract
  v = ((v - minVal)/maxVal) * 255


In [4]:
# take selected roi from corresponding directory
roi_dir = "E:\\Data\\sam_data\\new\\MD_1264_A16_Z3.3mm_corr_phrt\\roi"
selected_rois = os.listdir(roi_dir) 

In [5]:
# filtering rois according to the z stack 
grouped_rois = {}
for r in selected_rois:
    grp = r.split('x')[0]
    if grp in grouped_rois:
            grouped_rois[grp].append(r)
    else:
        grouped_rois[grp] = [r]
    

In [7]:
def visualize_selected_roi(arr, grp_roi, cmap='gray', **kwargs):
    """
    This function creates an interactive visualization of the input array in 2D or 3D.

    Parameters:
    arr (ndarray): The input array to be visualized.
    cmap (str): The colormap to use for the plot. Default is 'viridis'
    **kwargs: Additional keyword arguments to be passed to plt.imshow.

    Returns:
    None
    """
    select_roi_range = widgets.Dropdown(options=grp_roi.keys(), description='Select option:')

    def select_vis_range(vis_range):
        print(vis_range)

    
        slider = widgets.IntSlider(
            min=int(vis_range.split('-')[0]), max=int(vis_range.split('-')[1])-1, step=1, value=int(vis_range.split('-')[0]))

        def visualizeation(slice_number):
            """
            This function plots the 3D visualization of the input array.

            Parameters:
            slice_number (int): The slice number of the input array to be visualized.

            Returns:
            None
            """
            fig, ax = plt.subplots(figsize=(16,9))
            for aroi in grp_roi[vis_range]:
                roi_xrange = aroi.split('x')[2]
                x = roi_xrange.split('-')[0]
                roi_yrange = aroi.split('x')[1]
                y = roi_yrange.split('-')[0]

                rect = plt.Rectangle((int(x), int(y)), 300, 300, fill=False, edgecolor='r', lw=1)
                ax.add_patch(rect)

            # plt.subplots_adjust(hspace=0.1, wspace=0.1)
            ax.imshow(arr[slice_number, :, :], cmap=cmap, **kwargs)
            plt.show()


        widgets.interact(visualizeation, slice_number=slider)
    
    widgets.interact(select_vis_range, vis_range=select_roi_range)
    

In [8]:
visualize_selected_roi(vol, grouped_rois)

interactive(children=(Dropdown(description='Select option:', options=('0-300', '200-500', '400-700', '600-900'…